In [1]:
import sys
import os

# 设置路径
model_path = 'D:/Ti/Py_mmWave_Roformer/rope_informer'
processed_data_path = 'D:/Ti/Py_mmWave_Roformer/processed_data'

checkpoints = 'D:/Ti/Py_mmWave_Roformer/checkpoints/'
output_path = 'D:/Ti/Py_mmWave_Roformer/test_results/'

# 确保目录存在
os.makedirs(checkpoints, exist_ok=True)
os.makedirs(output_path, exist_ok=True)

# 将模型路径添加到sys.path
sys.path.append(os.path.dirname(model_path))

print("环境设置完成")

环境设置完成


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

# 为 NumPy 2.0 提供向后兼容性
if not hasattr(np, 'Inf'):
    np.Inf = np.inf

print("库导入完成")

库导入完成


In [4]:
# 加载处理好的数据
X = np.load(os.path.join(processed_data_path, "X_test.npy"))
y = np.load(os.path.join(processed_data_path, "y_test.npy"))

print(f"加载数据形状: X={X.shape}, y={y.shape}")


# 创建PyTorch数据集
class ImageSequenceDataset(Dataset):
    """图像序列数据集类"""

    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        # 获取图像序列
        sequence = torch.FloatTensor(self.X[idx])
        # 获取标签
        label = torch.tensor(self.y[idx], dtype=torch.long)
        return sequence, label


# 创建完整数据集
full_dataset = ImageSequenceDataset(X, y)

# 划分训练集和测试集
indices = list(range(len(full_dataset)))
train_indices, test_indices = train_test_split(
    indices, test_size=0.2, random_state=42, stratify=y
)

# 创建子集数据集
train_dataset = Subset(full_dataset, train_indices)
test_dataset = Subset(full_dataset, test_indices)

# 创建数据加载器
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

print(f"训练集大小: {len(train_dataset)}")
print(f"测试集大小: {len(test_dataset)}")
print("数据加载和预处理完成")

加载数据形状: X=(810, 5, 2, 25, 25), y=(810,)
训练集大小: 648
测试集大小: 162
数据加载和预处理完成


In [5]:
import argparse
from rope_informer import Exp_Informer

# 自定义实验类，处理图像序列数据
class Exp_Informer_Action(Exp_Informer):
    def __init__(self, args):
        super(Exp_Informer_Action, self).__init__(args)
        # 修改损失函数为交叉熵
        self.criterion = nn.CrossEntropyLoss()

    def _get_data(self, flag):
        """获取数据加载器"""
        if flag == 'test':
            return test_dataloader
        else:
            return train_dataloader

    def _process_one_batch(self, batch_x, batch_y, batch_x_mark, batch_y_mark):
        """处理一个批次的数据"""
        # 对于分类任务，我们不需要batch_x_mark和batch_y_mark
        # 将数据移动到设备
        batch_x = batch_x.float().to(self.device)
        batch_y = batch_y.long().to(self.device)

        # 展平图像序列 (batch_size, seq_len, channels, height, width) -> (batch_size, seq_len, channels*height*width)
        batch_size, seq_len, channels, height, width = batch_x.shape
        batch_x = batch_x.view(batch_size, seq_len, -1)

        # 前向传播
        if self.args.output_attention:
            outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
        else:
            outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)

        # 计算损失
        loss = self.criterion(outputs, batch_y)

        return outputs, loss

print("自定义实验类定义完成")

# 设置Informer参数
parser = argparse.ArgumentParser(description='[Informer] Action Recognition')

# 关键参数修改 - 根据您的数据调整
parser.add_argument('--model', type=str, default='informer', help='model of experiment')
parser.add_argument('--data', type=str, default='ActionRecognition', help='data name')
parser.add_argument('--root_path', type=str, default=processed_data_path, help='root path of the data file')
parser.add_argument('--enc_in', type=int, default=1250, help='encoder input size')  # 2*25*25=1250
parser.add_argument('--d_model', type=int, default=64, help='dimension of model')
parser.add_argument('--d_ff', type=int, default=256, help='dimension of fcn')
parser.add_argument('--train_epochs', type=int, default=20, help='train epochs')
parser.add_argument('--batch_size', type=int, default=batch_size, help='batch size of train input data')
parser.add_argument('--seq_len', type=int, default=5, help='input sequence length')  # 窗口大小
parser.add_argument('--output_path', type=str, default=output_path, help='test_results')
parser.add_argument('--checkpoints', type=str, default=checkpoints, help='location of model checkpoints')
parser.add_argument('--test_ratio', type=float, default=0.2, help='test ratio')
parser.add_argument('--n_heads', type=int, default=4, help='num of heads')
parser.add_argument('--has_rope', type=bool, default=True, help='use rope position encoding')

# 其他参数
parser.add_argument('--data_path', type=str, default='', help='data file')
parser.add_argument('--features', type=str, default='M', help='forecasting task')
parser.add_argument('--target', type=str, default='', help='target feature')
parser.add_argument('--freq', type=str, default='h', help='freq for time features encoding')
parser.add_argument('--label_len', type=int, default=3, help='start token length')  # 调整为适合分类任务
parser.add_argument('--pred_len', type=int, default=1, help='prediction sequence length')  # 分类任务预测长度为1
parser.add_argument('--dec_in', type=int, default=1250, help='decoder input size')
parser.add_argument('--c_out', type=int, default=5, help='output size')  # 5个类别
parser.add_argument('--e_layers', type=int, default=2, help='num of encoder layers')
parser.add_argument('--d_layers', type=int, default=1, help='num of decoder layers')
parser.add_argument('--s_layers', type=str, default='3,2,1', help='num of stack encoder layers')
parser.add_argument('--factor', type=int, default=5, help='probsparse attn factor')
parser.add_argument('--padding', type=int, default=0, help='padding type')
parser.add_argument('--distil', action='store_false', default=True, help='whether to use distilling in encoder')
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--attn', type=str, default='prob', help='attention used in encoder')
parser.add_argument('--embed', type=str, default='fixed', help='time features encoding')
parser.add_argument('--activation', type=str, default='gelu', help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in encoder')
parser.add_argument('--do_predict', action='store_true', help='whether to predict unseen future data')
parser.add_argument('--mix', action='store_false', default=True, help='use mix attention in generative decoder')
parser.add_argument('--cols', type=str, nargs='+', help='certain cols from the data files as the input features')
parser.add_argument('--num_workers', type=int, default=2, help='data loader num workers')
parser.add_argument('--itr', type=int, default=1, help='experiments times')
parser.add_argument('--patience', type=int, default=5, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test', help='exp description')
parser.add_argument('--loss', type=str, default='cross_entropy', help='loss function')  # 改为交叉熵损失
parser.add_argument('--lradj', type=str, default='type1', help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', default=False, help='use automatic mixed precision training')
parser.add_argument('--inverse', action='store_true', default=False, help='inverse output data')
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', default=False, help='use multiple gpus')
parser.add_argument('--devices', type=str, default='0', help='device ids of multile gpus')

args = parser.parse_args([])  # 传递空列表避免命令行参数错误

args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

if args.use_gpu and args.use_multi_gpu:
    args.devices = args.devices.replace(' ', '')
    device_ids = args.devices.split(',')
    args.device_ids = [int(id_) for id_ in device_ids]
    args.gpu = args.device_ids[0]

print('实验参数:')
for arg in vars(args):
    print(f"  {arg}: {getattr(args, arg)}")

print("模型参数配置完成")

自定义实验类定义完成
实验参数:
  model: informer
  data: ActionRecognition
  root_path: D:/Ti/Py_mmWave_Roformer/processed_data
  enc_in: 1250
  d_model: 64
  d_ff: 256
  train_epochs: 20
  batch_size: 16
  seq_len: 5
  output_path: D:/Ti/Py_mmWave_Roformer/test_results/
  checkpoints: D:/Ti/Py_mmWave_Roformer/checkpoints/
  test_ratio: 0.2
  n_heads: 4
  has_rope: True
  data_path: 
  features: M
  target: 
  freq: h
  label_len: 3
  pred_len: 1
  dec_in: 1250
  c_out: 5
  e_layers: 2
  d_layers: 1
  s_layers: 3,2,1
  factor: 5
  padding: 0
  distil: True
  dropout: 0.05
  attn: prob
  embed: fixed
  activation: gelu
  output_attention: False
  do_predict: False
  mix: True
  cols: None
  num_workers: 2
  itr: 1
  patience: 5
  learning_rate: 0.0001
  des: test
  loss: cross_entropy
  lradj: type1
  use_amp: False
  inverse: False
  use_gpu: False
  gpu: 0
  use_multi_gpu: False
  devices: 0
模型参数配置完成


In [ ]:
# 设置实验

torch.manual_seed(111)

# 实验设置
setting = '{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_at{}_fc{}_eb{}_dt{}_mx{}_{}_{}'.format(
    args.model, args.data, args.features,
    args.seq_len, args.label_len, args.pred_len,
    args.d_model, args.n_heads, args.e_layers, args.d_layers, args.d_ff, args.attn, args.factor,
    args.embed, args.distil, args.mix, args.des, 1)

exp = Exp_Informer_Action(args)  # 创建实验实例

# 训练模型
print('>>>>>>>开始训练: {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
exp.train(setting)

print("模型训练完成")

Use CPU
>>>>>>>开始训练: informer_ActionRecognition_ftM_sl5_ll3_pl1_dm64_nh4_el2_dl1_df256_atprob_fc5_ebfixed_dtTrue_mxTrue_test_1>>>>>>>>>>>>>>>>>>>>>>>>>>


In [ ]:
# 测试模型
print('>>>>>>>测试: {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
exp.test(setting)

# 清理GPU缓存
torch.cuda.empty_cache()

print("模型测试完成")

In [ ]:
# 加载测试结果并可视化
# 加载真实标签和预测结果
truth = np.load(os.path.join(args.output_path, setting, "true.npy"))
preds = np.load(os.path.join(args.output_path, setting, "pred.npy"))

# 计算每个类别的总数和预测正确的数量
bin_counts = np.bincount(truth, minlength=5)
correct_counts = np.bincount(truth[preds == truth], minlength=5)

# 绘制直方图
categories = range(5)
class_names = ['stationary', 'run', 'squat', 'stand', 'walk']

plt.figure(figsize=(10, 6))
plt.bar(categories, bin_counts, width=0.5, align='center', alpha=0.8, label='真实', color='purple')
plt.bar(categories, correct_counts, width=0.5, align='edge', alpha=1.0, label='正确', color='lightgreen')

# 添加数值标签
for i, (bin_count, correct_count) in enumerate(zip(bin_counts, correct_counts)):
    plt.text(i, bin_count + 0.1, str(bin_count), ha='center', va='bottom')
    plt.text(i, correct_count + 0.1, str(correct_count), ha='center', va='bottom')

plt.xlabel('类别')
plt.ylabel('数量')
plt.xticks(categories, class_names)
plt.legend()
plt.title('分类结果统计')

# 保存图像
plt.savefig('D:/Ti/Py_mmWave_Roformer/classification_results.png', dpi=300, bbox_inches='tight')
plt.show()

# 计算并显示准确率
accuracy = np.mean(preds == truth)
print(f"测试准确率: {accuracy:.4f}")

# 计算每个类别的准确率
class_accuracies = []
for i in range(5):
    class_mask = truth == i
    if np.sum(class_mask) > 0:
        class_accuracy = np.mean(preds[class_mask] == truth[class_mask])
        class_accuracies.append(class_accuracy)
        print(f"{class_names[i]} 准确率: {class_accuracy:.4f}")

# 绘制类别准确率
plt.figure(figsize=(10, 6))
plt.bar(range(5), class_accuracies, color='skyblue')
plt.xlabel('类别')
plt.ylabel('准确率')
plt.xticks(range(5), class_names)
plt.title('各类别准确率')
plt.ylim(0, 1)

# 添加数值标签
for i, acc in enumerate(class_accuracies):
    plt.text(i, acc + 0.01, f'{acc:.3f}', ha='center', va='bottom')

plt.savefig('D:/Ti/Py_mmWave_Roformer/class_accuracy.png', dpi=300, bbox_inches='tight')
plt.show()

print("结果可视化和分析完成")